### 最適化モデル作成の高度なテクニック

In [2]:
import numpy as np
from pulp import *
from ortoolpy import *

In [3]:
m=LpProblem() # 数理モデル
x1=addvars(10) # 一次元変数
x= addvar() # 0次元変数
z= addbinvar() # 0ー1変数
m += lpSum(x) # 目的関数

#### 変数をnp.arrayで作成する：Numpyのスライスなどの機能が使える

In [4]:
x2=addvars(2,3) # 2行3列の二次元の変数(リスト)
x3=np.array(x2) # 2行3列の二次元の変数(numpy)

In [5]:
x2

[[v000013, v000014, v000015], [v000016, v000017, v000018]]

In [6]:
x3

array([[v000013, v000014, v000015],
       [v000016, v000017, v000018]], dtype=object)

#### 結果をnp.vectorizeで取得する：内包表記より簡潔に記述でき、高速に動作する

In [9]:
# 内包表記
val = [value(v) for v in x1]
val

[None, None, None, None, None, None, None, None, None, None]

In [12]:
# np.vaectorize
val=np.vectorize(value)(x1)
val

array([None, None, None, None, None, None, None, None, None, None],
      dtype=object)

#### 変数の範囲を指定する：範囲が記述できる

In [15]:
# 制約条件で指定する場合
m += x >= 1
m += x <= 3
# 変数作成時に指定する場合
x=LpVariable('x',lowBound=1,upBound=3)

In [16]:
# 半連続変数：停止(0)とある上下限(10から100)しか許されていない場合
# 0-1変数を使って記述
m += x>= 10*z
m += x<= 100*z 

In [18]:
# 2つに分かれた範囲(10以下か20以上)のどちらかしか取れない
# 0-1変数と十分大きな数(M)を使って記述
M=100 # サンプル
m+=x <= 10+M*z # z=0の時有効条件、1の時無効条件
m+=x>=20-M*(1-z)

#### 変数リスト内の変数を全て同じ値とする制約条件をしてする：nこのペアはn(n-1)/2だけであるが、n-1個の制約条件を指定するだけで済む

In [19]:
for vi, vj in zip(x1,x1[1:]):
    m+=vi==vj

#### リストの中のどれか1つを選ぶように制限する：制約条件を記述できる

In [21]:
 m += lpSum(x1) ==1

#### リストの合計が1になるのを禁止する：制約条件を記述できる

In [23]:
for v in x1:
    m += 2*v <= lpSum(x1)

#### 隣接制約を指定する：制約条件を記述できる
- 隣接制約：2つの条件のうち、1つだけ満たしていれば良い

In [26]:
for x in [0,1,2,3,4]:
    m=LpProblem()
    y=addvar(lowBound=None)
    m+=y # 目的関数（yの最小化）
    m+=y>=2-x
    m+=y>=-2-x
    m.solve()
    print(f'x,y={x},{value(y)}')

x,y=0,2.0
x,y=1,1.0
x,y=2,0.0
x,y=3,-1.0
x,y=4,-2.0


In [27]:
# 隣接制約
M=4 # 十分大きな数
for x in [0,1,2,3,4]:
    m=LpProblem()
    y=addvar(lowBound=None)
    z=addbinvar() # 0-1変数
    m += y
    m += y >= 2-x-M*z
    m += y >= -2 + x - M * (1-z)
    m.solve()
    print(f'x,y={x},{value(y)}')    

x,y=0,-2.0
x,y=1,-1.0
x,y=2,0.0
x,y=3,-1.0
x,y=4,-2.0


#### く